In [1]:
# RL deconvolution function adapted from skimage.restoration.richardson_lucy
# https://scikit-image.org/docs/stable/auto_examples/filters/plot_deconvolution.html
# accessed on 12.06.2023

import numpy as np
import numpy.random as npr
from scipy.signal import convolve
from astropy.io import fits
from PIL import Image
from astropy.utils.data import get_pkg_data_filename
import matplotlib.pyplot as plt
from skimage import restoration
from astropy.convolution import Gaussian2DKernel
import torch 
import glob
import pandas as pd
import os

import sys
sys.path.append("../Data Preparation")  # locally defined
from utils_custom import calc_psnr, calc_ssim, calc_sssim # modify utils to utils_custom to avoid conflict with Python package 'utils'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
### 
#唯一需要修改的地方！！！
###
metric='sssim' # IMPORTANT: choose the metric of evaluating similarity between hr and lr here, 'ssim' or 'psnr' or 'sssim'
change='10_0015' # IMPORTANT: choose the change from hr to lr here, '10_0015' or '10_001' or '4_001'
psf=np.array(Gaussian2DKernel(x_stddev=10)) # IMPORTANT: choose the psf for RL method, '10' or '4' the same as the change from hr to lr

In [3]:
image_dataset_name = 'Hubble_Images_top100'
result_data_folder_name = 'result_data_se_'+image_dataset_name+'/'
result_plot_folder_name = 'result_plot_se_'+image_dataset_name+'/'

In [4]:
se_hr_dir = '../../Prep_Images/'+image_dataset_name+'_se_hr_'+change # images from preparation step
se_lr_dir = '../../Prep_Images/'+image_dataset_name+'_se_lr_'+change 

rl_dir = '../../RL_Images/'+image_dataset_name+'_'+metric+'_'+change

In [5]:
# for TIFF
def richardson_lucy(image_lr, image_hr, metric='sssim', psf=np.array(Gaussian2DKernel(x_stddev=10)), iterations=50, clip=True, filter_epsilon=None):
    """Richardson-Lucy deconvolution.

    Parameters
    ----------
    image : ndarray
       Input degraded image (can be N dimensional).
    psf : ndarray
       The point spread function.
    iterations : int, optional
       Number of iterations. This parameter plays the role of
       regularisation.
    clip : boolean, optional
       True by default. If true, pixel value of the result above 1 or
       under -1 are thresholded for skimage pipeline compatibility.
    filter_epsilon: float, optional
       Value below which intermediate results become 0 to avoid division
       by small numbers.

    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Richardson%E2%80%93Lucy_deconvolution
    """
    # 指标函数映射
    metric_map = {
        "sssim": calc_sssim,
        "ssim": calc_ssim,
        "psnr": calc_psnr
    }

    if metric not in metric_map:
        raise ValueError(f"Unsupported metric '{metric}'. Choose from: {list(metric_map.keys())}")

    # metric_fn = metric_map[metric]


    float_type = np.promote_types(image_lr.dtype, np.float32)
    image_lr = image_lr.astype(float_type, copy=False)
    psf = psf.astype(float_type, copy=False)
    im_deconv = np.full(image_lr.shape, 0.5, dtype=float_type)
    psf_mirror = np.flip(psf)
    np.random.seed(123)

    best_score = 0
    for i in range(iterations):
        conv = convolve(im_deconv, psf, mode='same')
        if filter_epsilon:
            relative_blur = np.where(conv < filter_epsilon, 0, image_lr / conv)
        else:
            relative_blur = image_lr / conv

        im_deconv *= convolve(relative_blur, psf_mirror, mode='same')

        # sssim_results[i, file_ind] = calc_sssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        # ssim_results[i, file_ind] = calc_ssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        # psnr_results[i, file_ind] = calc_psnr(img1=torch.Tensor(image_hr), img2=torch.Tensor(im_deconv))
        if metric == 'sssim':
            res_metric = calc_sssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        if metric == 'ssim':
            res_metric = calc_ssim(img1=torch.Tensor(image_hr.reshape(1, image_hr.shape[0], image_hr.shape[1])), img2=torch.Tensor(im_deconv.reshape(1, im_deconv.shape[0], im_deconv.shape[1])))
        if metric == 'psnr':
            res_metric = calc_psnr(img1=torch.Tensor(image_hr), img2=torch.Tensor(im_deconv))
        

        if res_metric > best_score:
            best_score = res_metric
            epoch_no = i
            # im_deconv_best = im_deconv
            im_deconv_best = im_deconv.copy()


    if clip:
        im_deconv_best[im_deconv_best > 1] = 1
        im_deconv_best[im_deconv_best < -1] = -1  

    return best_score, epoch_no, im_deconv_best

In [6]:
# main

np.random.seed(123)
ind = 0
metric_list = []

os.makedirs(rl_dir, exist_ok=True)

for filename in glob.glob(os.path.join(se_hr_dir, "*.tif")):
    ind += 1

    img_hr = Image.open(filename)
    imarray_hr = np.array(img_hr).astype(np.float32)

    filename_lr = os.path.join(se_lr_dir, 'lr_' + filename.split("/")[-1])

    img_lr = Image.open(filename_lr)
    imarray_lr = np.array(img_lr).astype(np.float32)

    rl_filename = os.path.join(rl_dir, 'rl_' + filename.split("/")[-1])

    best_score, epoch_no, im_deconv_best = richardson_lucy(image_lr=imarray_lr,image_hr=imarray_hr, metric=metric, psf=psf)

    # hdu = fits.PrimaryHDU(im_deconv_best)
    # hdu.writeto(rl_filename, overwrite=True)

    Image.fromarray(im_deconv_best).save(rl_filename)
    
    # Append results to the list
    metric_list.append({
        "file_index": ind,
        "best_score": float(best_score.item()),
        "epoch_no": epoch_no
    })

    print(ind)


rl_df = pd.DataFrame(metric_list)
rl_df.to_csv(result_data_folder_name+metric+'_'+change+'.csv')

print(rl_df['best_score'].mean())
print(rl_df['epoch_no'].mean())

1
2
3
4
5
6
7


/var/folders/n2/h_p1g87j4fx1l3clf8fzzls80000gn/T/ipykernel_24771/449095905.py:51: RuntimeWarning: invalid value encountered in divide
  relative_blur = image_lr / conv
/var/folders/n2/h_p1g87j4fx1l3clf8fzzls80000gn/T/ipykernel_24771/449095905.py:53: RuntimeWarning: Use of fft convolution on input with NAN or inf results in NAN or inf output. Consider using method='direct' instead.
  im_deconv *= convolve(relative_blur, psf_mirror, mode='same')


ValueError: Input X contains NaN.
KMeans does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values